<a href="https://colab.research.google.com/github/SpandanaKalakonda/Building_ML_Pipelines/blob/main/data_validation_using_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tfx

In [ ]:
!pip install apache-beam

In [ ]:
!git clone https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines.git

In [67]:
%cd building-machine-learning-pipelines/
!python3 utils/download_dataset.py
%cd ..

/content/building-machine-learning-pipelines
INFO:root:Started download script
INFO:root:Data folder already existed.
INFO:urllib3.poolmanager:Redirecting http://bit.ly/building-ml-pipelines-dataset -> https://drive.google.com/uc?export=download&id=1VHjb8L8n2d6eLz_lA-F-bk6Z0UecHpEF
INFO:urllib3.poolmanager:Redirecting https://drive.google.com/uc?export=download&id=1VHjb8L8n2d6eLz_lA-F-bk6Z0UecHpEF -> https://doc-0o-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/f94d1rangq9bqd1rkdqqe5454opaprel/1701387075000/06616860426990197454/*/1VHjb8L8n2d6eLz_lA-F-bk6Z0UecHpEF?e=download&uuid=f999e74f-ec59-4c27-8571-b114b232c44f
INFO:root:Download completed.
INFO:root:Finished download script
/content


In [68]:
import tensorflow_data_validation as tfdv
import csv
import tensorflow as tf
from tqdm import tqdm

In [69]:
stats = tfdv.generate_statistics_from_csv(
    data_location='/content/building-machine-learning-pipelines/data/consumer_complaints_with_narrative.csv',
    delimiter=','
    )

In [70]:
tfdv.visualize_statistics(stats)

In [71]:
schema = tfdv.infer_schema(stats)

In [72]:
schema

feature {
  name: "product"
  type: BYTES
  domain: "product"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sub_product"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_product"
  presence {
    min_count: 1
  }
}
feature {
  name: "issue"
  type: BYTES
  domain: "issue"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sub_issue"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_issue"
  presence {
    min_count: 1
  }
}
feature {
  name: "consumer_complaint_narrative"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "company"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "state"
  value_count {
    min: 1
    max: 1
  }
  type:

In [73]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'product',STRING,required,,'product'
'sub_product',STRING,optional,single,'sub_product'
'issue',STRING,required,,'issue'
'sub_issue',STRING,optional,single,'sub_issue'
'consumer_complaint_narrative',BYTES,required,,-
'company',BYTES,required,,-
'state',STRING,optional,single,'state'
'zip_code',BYTES,optional,single,-
'company_response',STRING,required,,'company_response'


,Values
Domain,
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'sub_product',"'(CD) Certificate of deposit', 'Auto', 'Cashing a check without an account', 'Check cashing', 'Checking account', 'Conventional adjustable mortgage (ARM)', 'Conventional fixed mortgage', 'Credit card', 'Credit repair', 'Debt settlement', 'Domestic (US) money transfer', 'Electronic Benefit Transfer / EBT card', 'FHA mortgage', 'Federal student loan', 'Foreign currency exchange', 'General purpose card', 'Gift or merchant card', 'Government benefit payment card', 'Home equity loan or line of credit', 'I do not know', 'ID prepaid card', 'Installment loan', 'International money transfer', 'Medical', 'Mobile wallet', 'Money order', 'Mortgage', 'Non-federal student loan', 'Other (i.e. phone, health club, etc.)', 'Other bank product/service', 'Other mortgage', 'Other special purpose card', 'Pawn loan', 'Payday loan', 'Payroll card', 'Personal line of credit', 'Refund anticipation check', 'Reverse mortgage', 'Savings account', 'Title loan', 'Transit card', 'Travelerâs/Cashierâs checks', 'VA mortgage', 'Vehicle lease', 'Vehicle loan'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 

In [74]:
# Save the schema to a file
tfdv.write_schema_text(schema, 'schema_csv.pbtxt')

In [75]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [76]:
def clean_rows(row):
  if not row["zip_code"]:
    row["zip_code"] = "99999"
  return row

In [77]:
def convert_zipcode_to_int(zipcode):
  if isinstance(zipcode, str) and "XX" in zipcode:
    zipcode = zipcode.replace("XX", "00")
  int_zipcode = int(zipcode)
  return int_zipcode

In [78]:
original_data_file = "/content/building-machine-learning-pipelines/data/consumer_complaints_with_narrative.csv"
tfrecord_filename = "consumer_complaints.tfrecord"
tf_record_writer = tf.io.TFRecordWriter(tfrecord_filename)

In [79]:
with open(original_data_file) as csv_file:
  reader = csv.DictReader(csv_file, delimiter=",", quotechar='"')
  for row in tqdm(reader):
    row = clean_rows(row)
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "product": _bytes_feature(row["product"]),
                "sub_product": _bytes_feature(row["sub_product"]),
                "issue": _bytes_feature(row["issue"]),
                "sub_issue": _bytes_feature(row["sub_issue"]),
                "state": _bytes_feature(row["state"]),
                "zip_code": _int64_feature(convert_zipcode_to_int(row["zip_code"])),
                "company": _bytes_feature(row["company"]),
                "company_response": _bytes_feature(row["company_response"]),
                "consumer_complaint_narrative": _bytes_feature(row["consumer_complaint_narrative"]),
                "timely_response": _bytes_feature(row["timely_response"]),
                "consumer_disputed": _bytes_feature(row["consumer_disputed"]),
            }
        )
        )
    tf_record_writer.write(example.SerializeToString())
  tf_record_writer.close()

66799it [00:12, 5397.64it/s]


In [80]:
stats_tf_record = tfdv.generate_statistics_from_tfrecord(
    data_location='/content/consumer_complaints.tfrecord'
)

In [81]:
tfdv.visualize_statistics(stats_tf_record)

In [82]:
schema_tf_record = tfdv.infer_schema(stats_tf_record)

In [83]:
tfdv.display_schema(schema_tf_record)

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'consumer_disputed',BYTES,required,,-
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,required,,'state'
'sub_issue',STRING,required,,'sub_issue'
'sub_product',STRING,required,,'sub_product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'state',"'', 'AA', 'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'sub_issue',"'', 'Account status', 'Account terms', 'Account terms and changes', 'Applied for loan/did not receive money', 'Attempted to collect wrong amount', 'Attempted to/Collected exempt funds', 'Billing dispute', 'Called after sent written cease of comm', 'Called outside of 8am-9pm', 'Can\'t contact lender', 'Can\'t decrease my monthly payments', 'Can\'t get flexible payment options', 'Can\'t qualify for a loan', 'Can\'t stop charges to bank account', 'Can\'t temporarily postpone payments', 'Charg

In [84]:
# Save the schema to a file
tfdv.write_schema_text(schema_tf_record, 'schema_tf_record.pbtxt')

In [85]:
tfdv.visualize_statistics(lhs_statistics=stats, rhs_statistics=stats_tf_record,
                          lhs_name="CSV Dataset", rhs_name="TF RECORD Dataset")

In [86]:
anamolies = tfdv.validate_statistics(statistics=stats, schema=schema_tf_record)

In [87]:
tfdv.display_anomalies(anamolies)

,Anomaly short description,Anomaly long description
Feature name,,
'consumer_disputed',Unexpected data type,Expected data of type: BYTES but got INT
'sub_product',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.693828"
'zip_code',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.997171 Expected data of type: INT but got STRING"
'state',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.997216"
'sub_issue',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.492986"


In [88]:
anamolies

baseline {
  feature {
    name: "company"
    type: BYTES
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "company_response"
    type: BYTES
    domain: "company_response"
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "consumer_complaint_narrative"
    type: BYTES
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "consumer_disputed"
    type: BYTES
    bool_domain {
      true_value: "1"
      false_value: "0"
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "issue"
    type: BYTES
    domain: "issue"
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1

In [89]:
schema_tf = tfdv.load_schema_text("/content/schema_tf_record.pbtxt")

In [90]:
type(schema_tf)

tensorflow_metadata.proto.v0.schema_pb2.Schema

In [91]:
sub_issue_feature = tfdv.get_feature(schema_tf, 'sub_issue')
sub_issue_feature.presence.min_fraction = 0.9

In [92]:
state_domain = tfdv.get_domain(schema_tf, 'state')

In [93]:
state_domain

name: "state"
value: ""
value: "AA"
value: "AE"
value: "AK"
value: "AL"
value: "AP"
value: "AR"
value: "AS"
value: "AZ"
value: "CA"
value: "CO"
value: "CT"
value: "DC"
value: "DE"
value: "FL"
value: "FM"
value: "GA"
value: "GU"
value: "HI"
value: "IA"
value: "ID"
value: "IL"
value: "IN"
value: "KS"
value: "KY"
value: "LA"
value: "MA"
value: "MD"
value: "ME"
value: "MI"
value: "MN"
value: "MO"
value: "MP"
value: "MS"
value: "MT"
value: "NC"
value: "ND"
value: "NE"
value: "NH"
value: "NJ"
value: "NM"
value: "NV"
value: "NY"
value: "OH"
value: "OK"
value: "OR"
value: "PA"
value: "PR"
value: "RI"
value: "SC"
value: "SD"
value: "TN"
value: "TX"
value: "UT"
value: "VA"
value: "VI"
value: "VT"
value: "WA"
value: "WI"
value: "WV"
value: "WY"

In [94]:
state_domain.value.remove('AK')

In [95]:
state_domain

name: "state"
value: ""
value: "AA"
value: "AE"
value: "AL"
value: "AP"
value: "AR"
value: "AS"
value: "AZ"
value: "CA"
value: "CO"
value: "CT"
value: "DC"
value: "DE"
value: "FL"
value: "FM"
value: "GA"
value: "GU"
value: "HI"
value: "IA"
value: "ID"
value: "IL"
value: "IN"
value: "KS"
value: "KY"
value: "LA"
value: "MA"
value: "MD"
value: "ME"
value: "MI"
value: "MN"
value: "MO"
value: "MP"
value: "MS"
value: "MT"
value: "NC"
value: "ND"
value: "NE"
value: "NH"
value: "NJ"
value: "NM"
value: "NV"
value: "NY"
value: "OH"
value: "OK"
value: "OR"
value: "PA"
value: "PR"
value: "RI"
value: "SC"
value: "SD"
value: "TN"
value: "TX"
value: "UT"
value: "VA"
value: "VI"
value: "VT"
value: "WA"
value: "WI"
value: "WV"
value: "WY"

In [96]:
tfdv.write_schema_text(schema_tf, "/content/schema_tf_record.pbtxt")

In [97]:
updated_anomalies = tfdv.validate_statistics(statistics=stats, schema=schema_tf)

In [98]:
tfdv.display_anomalies(updated_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'zip_code',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.997171 Expected data of type: INT but got STRING"
'consumer_disputed',Unexpected data type,Expected data of type: BYTES but got INT
'sub_product',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.693828"
'state',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.997216 Examples contain values missing from the schema: AK (<1%)."
'sub_issue',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 0.900000, actual = 0.492986"
